In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,transform_alocation_dataframe,base_selection,expand_rows,replicate_row,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler
from streamlit_app.validador import validador

In [4]:
aulas_raw = pd.read_excel('/home/cayena/Downloads/ROTA_TESTE_VALID.xlsx')
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final(4).xlsx')


In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [6]:
validador(df_result,df_professor).check_problem()

Teachers not found in the teacher table: {'Allan'}
Hours not found in the teacher table: {'23:59:00'} for the classes: ['CONVERSATION 05 EAD - 2X']
Horários não permitidos para os professores:
{'LUCIANO': ['16:00:00'], 'ANDREY': ['08:00:00', '09:00:00', '10:00:00']}
Dia da semana não permitidos para os professores:
{'DUDU': ['TERÇA']}
Teacher BRUNO MORGILLO has multiple classes in the same hour: ['CURAÇAU EAD - 4X' 'RENATO VIP CONV EAD - 1X']
Teacher BRUNO MORGILLO has multiple classes in the same hour: ['CURAÇAU EAD - 4X' 'KIKO VIP P - 1X']
Teacher MARIA has multiple classes in the same hour: ['ERILENE VIP EAD - 2X' 'GEISSE VIP CONV EAD - 1X']
Teacher MARIA has multiple classes in the same hour: ['ROMARIO E MARILENE VIP EAD - 2X' 'MONTERREY EAD - 3X']
Teacher RODRIGO has multiple classes in the same hour: ['EDINBURGH P - 3X' 'SUNDERLAND P - 3X']
Professor ANA - MBA tem turmas com diferença menor que 1 hora no dia da semana: SEGUNDA
Teacher ANDREY cannot teach in status PRESENCIAL
Days

In [7]:
Ts = TeacherScheduler(df_result, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada

KeyError: 'eSat'

In [9]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,livro,modalidade,grupo,n aulas,parag atual grupo,...,rescisao,permuta,bolsista,n_total_alunos,stage_formula,parag_final_formula,num_1_sempre,ultimo_professor,penultimo_professor,restricoes_professor
